In [1]:
!pip install optuna
!pip install optuna-dashboard
!pip install optuna-integration


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from tqdm import tqdm

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost.core import XGBoostError
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


import csv

import cupy as cp

import gc

In [3]:
df = pd.read_csv("../../story_dataset.csv")
df

,prompt_id,prompt,story,hidden_state_file,len_generated_story
0,1,Once upon a time there was a dragon,Once upon a time there was a dragon. It was bi...,./hidden_states/prompt_1.npz,273
1,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He loved ...,./hidden_states/prompt_1.npz,246
2,1,Once upon a time there was a dragon,Once upon a time there was a dragon named Sam....,./hidden_states/prompt_1.npz,397
3,1,Once upon a time there was a dragon,Once upon a time there was a dragon. He was a ...,./hidden_states/prompt_1.npz,294
4,1,Once upon a time there was a dragon,Once upon a time there was a dragon. The drago...,./hidden_states/prompt_1.npz,296
...,...,...,...,...,...
9995,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,315
9996,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,270
9997,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,206
9998,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,375


In [4]:
hidden_states_by_layer = {}
NUM_PROMPTS = 10
# NUM_PROMPTS = 2

#Load hidden states to generate first token
for prompt_id in range(1, NUM_PROMPTS + 1):
    with np.load(f'../../../llamatales-xgboost-ii/hidden_states/first_prompt_{prompt_id}.npz') as loaded_data:
        for i in tqdm(range(1000)):
            curr_hidden_states = loaded_data[f"arr_{i}"]
            # print(curr_hidden_states.shape)
            
            #By layer
            for layer in range(1):
                curr_layer_hidden_states = curr_hidden_states[layer][0].astype('float32') #FAISS expects data in type float32 instead of float64 - saves memory too!
                # print(curr_layer_hidden_states.shape)
                
                if(f"layer_{layer}" in hidden_states_by_layer):
                    hidden_states_by_layer[f"layer_{layer}"].append(curr_layer_hidden_states)
                else:
                    hidden_states_by_layer[f"layer_{layer}"] = [curr_layer_hidden_states]

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1098.58it/s]


In [5]:
layer_hs_array = hidden_states_by_layer["layer_0"]
print(layer_hs_array[0].shape)
print(layer_hs_array[-1].shape)
#9x512 (prompts 1-9) or 10x512 (prompt 10)

#Convert prompt 10 hidden states to only encode last 9 tokens so dimensions are consistent w other prompts
for hs in range(len(layer_hs_array)):
    if(layer_hs_array[hs].shape[0] > 9):
        layer_hs_array[hs] = layer_hs_array[hs][1:]


for hs in range(len(layer_hs_array)):
    layer_hs_array[hs] = layer_hs_array[hs].flatten()

(9, 512)
(10, 512)


In [6]:
min_story_len = min(df["len_generated_story"])
min_story_len

37

In [7]:
max_story_len = max(df[:NUM_PROMPTS*1000]["len_generated_story"])

In [8]:
df[df["len_generated_story"] >= max_story_len]

,prompt_id,prompt,story,hidden_state_file,len_generated_story
9851,10,Once upon a time there was a poor boy,Once upon a time there was a poor boy named Ti...,./hidden_states/prompt_10.npz,522


In [9]:
def safe_split(total, train_ratio=0.8):
    train = int(train_ratio * total)   # floor
    test  = total - train              # leftover
    return train, test

def build_dataset(curr_context_level_hs, curr_labels, train_ratio=0.8):
    """
    Groups samples by prompt_id, splits safely into train/test,
    and returns X_train, y_train, X_test, y_test.
    """

    unique_ids = sorted(set(curr_labels))

    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []

    for pid in unique_ids:

        # get samples for this prompt id
        mask = (curr_labels == pid)
        X_pid = curr_context_level_hs[mask]
        y_pid = curr_labels[mask]

        total = len(X_pid)
        train_n, test_n = safe_split(total, train_ratio)

        # split
        X_train_list.append(X_pid[:train_n])
        y_train_list.append(y_pid[:train_n])

        X_test_list.append(X_pid[train_n:])
        y_test_list.append(y_pid[train_n:])

    # concatenate all prompt-id blocks
    X_train = np.concatenate(X_train_list, axis=0)
    y_train = np.concatenate(y_train_list, axis=0)
    X_test  = np.concatenate(X_test_list, axis=0)
    y_test  = np.concatenate(y_test_list, axis=0)

    # return cp.array(X_train), cp.array(y_train), cp.array(X_test), cp.array(y_test)
    return X_train, y_train, X_test, y_test
    

In [10]:
curr_labels = df.prompt_id.to_numpy() - 1
print(curr_labels)
curr_context_level_hs = np.array(layer_hs_array)

unique_ids = sorted(set(curr_labels))

X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_labels)
# X_train, y_train, X_test, y_test = train_test_split(curr_context_level_hs, curr_labels, test_size = 0.2, random_state=42)

# X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

X_train = cp.array(X_train)
y_train = cp.array(y_train)
X_test = cp.array(X_test)
y_test = cp.array(y_test)
# X_valid = cp.array(X_valid)
# y_valid = cp.array(y_valid)

# Defining the objective function
def objective(trial):
    param = {
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 0, 5),
    }
    
    # initializing the XGBoost model
    model = xgb.XGBClassifier(**param,
                              seed = 42, #for reproducibility
                              objective = 'multi:softmax',
                              eval_metric = "merror",
                              num_class = 10,
                              tree_method = 'hist',  # Using GPU for training 
                              device = 'cuda')       # If you have multiple GPUs, you can specify the GPU ID
    model.fit(X_train, y_train)

    # pruning_callback = XGBoostPruningCallback(trial, "validation_0-merror")
    
    # model.fit(
    #     X_train, y_train,
    #     eval_set=[(X_valid, y_valid)],
    #     callbacks=[pruning_callback],
    #     verbose=False
    # )

    # preds = model.predict(X_valid)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test.get(), preds)
    return accuracy

study = optuna.create_study(direction="maximize")
# study = optuna.create_study(
#     pruner=optuna.pruners.MedianPruner(), 
#     direction="maximize"
# )
study.optimize(objective, n_trials=100, show_progress_bar=True) # n_jobs=-1


print(f"Best trial: {study.best_trial.value}")
print(f"Best params: {study.best_params}")

[0 0 0 ... 9 9 9]


[I 2026-02-04 03:25:54,642] A new study created in memory with name: no-name-9b1f0c91-d112-41a2-a9d9-507fab6ff951


  0%|          | 0/100 [00:00<?, ?it/s]

[I 2026-02-04 03:26:02,394] Trial 0 finished with value: 1.0 and parameters: {'max_depth': 8, 'learning_rate': 0.09421898669488918, 'n_estimators': 612, 'subsample': 0.7748534514042495, 'colsample_bytree': 0.9063197390403603, 'min_child_weight': 6, 'gamma': 1.4792150096615457}. Best is trial 0 with value: 1.0.
[I 2026-02-04 03:26:12,259] Trial 1 finished with value: 1.0 and parameters: {'max_depth': 10, 'learning_rate': 0.09620331522778179, 'n_estimators': 832, 'subsample': 0.782842523375292, 'colsample_bytree': 0.6016799409595499, 'min_child_weight': 7, 'gamma': 0.4792344521911679}. Best is trial 0 with value: 1.0.
[I 2026-02-04 03:26:18,038] Trial 2 finished with value: 1.0 and parameters: {'max_depth': 6, 'learning_rate': 0.02314632669309201, 'n_estimators': 428, 'subsample': 0.5802256430399488, 'colsample_bytree': 0.5731198162219382, 'min_child_weight': 1, 'gamma': 4.793281347760296}. Best is trial 0 with value: 1.0.
[I 2026-02-04 03:26:22,825] Trial 3 finished with value: 1.0 and 

KeyboardInterrupt: 

In [9]:
#Context-level analysis
# min_story_len = min(df["len_new_story"])
# with open("results.csv", "w+", newline='') as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=',')
#     csv_writer.writerow(['Context Level', 'Accuracy'])

with open("../results.csv", "w+", newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    header = ['Layer', 'Context_Level', 'Train_Accuracy', 'Test_Accuracy']
    prompt_headers = []
    for i in range(1, 11):
        prompt_headers.extend([f"Prompt_{i}_Train_Accuracy", f"Prompt_{i}_Test_Accuracy"])

    header.extend(prompt_headers)
    csv_writer.writerow(header)

curr_labels = df.prompt_id.to_numpy() - 1
print(curr_labels)
curr_context_level_hs = np.array(layer_hs_array)

unique_ids = sorted(set(curr_labels))

X_train, y_train, X_test, y_test = build_dataset(curr_context_level_hs, curr_labels)
# X_train, y_train, X_test, y_test = train_test_split(curr_context_level_hs, curr_labels, test_size = 0.2, random_state=42)

X_train, y_train, X_valid, y_valid = train_test_split(X_train, y_train, test_size = 0.2, random_state=42)

# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# from sklearn.decomposition import PCA
# pca = PCA(n_components=0.95)

# X_train_pca = pca.fit_transform(X_train_scaled)
# X_test_pca = pca.transform(X_test_scaled)

# print("Train Data Shape: ", X_train_pca.shape)
# # print("Train Labels Shape: ", y_train.shape)
# print("Test Data Shape: ", X_test_pca.shape)
# # print("Test Labels Shape: ", y_test.shape)

# X_train_pca = cp.array(X_train_pca)
# y_train = cp.array(y_train)
# X_test_pca = cp.array(X_test_pca)
# y_test = cp.array(y_test)

print("Train Data Shape: ", X_train.shape)
#     print("Train Labels Shape: ", y_train.shape)
print("Test Data Shape: ", X_test.shape)
#     print("Test Labels Shape: ", y_test.shape)

X_train = cp.array(X_train)
y_train = cp.array(y_train)
X_test = cp.array(X_test)
y_test = cp.array(y_test)

# classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids))
# classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = len(unique_ids), tree_method='gpu_hist', predictor='gpu_predictor')
classifier = XGBClassifier(seed = 42, objective = 'multi:softmax', eval_metric = "merror", num_class = 10, tree_method='hist', device='cuda')
classifier.fit(X_train, y_train)
preds_train = classifier.predict(X_train)
preds = classifier.predict(X_test)

# classifier.fit(X_train_pca, y_train)
# preds = classifier.predict(X_test_pca)


#     print("Predictions Shape: ", preds.shape)

#     print("Unique Preds: ", pd.Series(preds).value_counts())
#     print("Unique Test Labels: ", pd.Series(y_test).value_counts())

train_accuracy = np.mean(cp.array(preds_train) == y_train)
accuracy = np.mean(cp.array(preds) == y_test)

print(f"Train Accuracy: {train_accuracy}")
print(f"Test Accuracy: {accuracy}")
# with open("results.csv", "a+", newline='') as csvfile:
#     csv_writer = csv.writer(csvfile, delimiter=',')
#     csv_writer.writerow([X_train.shape[1] // 512, accuracy])

prompt_accs = []

for i in range(10):
    mask = y_test == i
    prompt_test = y_test[mask]
    prompt_preds = cp.array(preds)[mask]
    prompt_test_acc = np.mean(prompt_preds == prompt_test)

    mask_train = y_train == i
    prompt_train = y_train[mask_train]
    prompt_preds_train = cp.array(preds_train)[mask_train]
    prompt_train_acc = np.mean(prompt_preds_train == prompt_train)

    print(f"Prompt {i + 1} Train Accuracy: {prompt_train_acc}")
    print(f"Prompt {i + 1} Test Accuracy: {prompt_test_acc}")

    prompt_accs.append(prompt_train_acc)
    prompt_accs.append(prompt_test_acc)


with open("../results.csv", "a+", newline='') as csvfile:
    csv_writer = csv.writer(csvfile, delimiter=',')
    values = [layer, 1, train_accuracy, accuracy]
    values.extend(prompt_accs)
    csv_writer.writerow(values)


del classifier, X_train, y_train, X_test, y_test, preds

# XGBoost cleanup
try:
    booster = classifier.get_booster()
    del booster
except:
    pass

# CuPy cleanup (CRITICAL)
cp.get_default_memory_pool().free_all_blocks()
cp.get_default_pinned_memory_pool().free_all_blocks()

gc.collect()


# print(max(curr_hs_shapes))

[0 0 0 ... 9 9 9]
Train Data Shape:  (8000, 4608)
Test Data Shape:  (2000, 4608)
Train Accuracy: 1.0
Accuracy: 1.0
Prompt 1 Train Accuracy: 1.0
Prompt 1 Test Accuracy: 1.0
Prompt 2 Train Accuracy: 1.0
Prompt 2 Test Accuracy: 1.0
Prompt 3 Train Accuracy: 1.0
Prompt 3 Test Accuracy: 1.0
Prompt 4 Train Accuracy: 1.0
Prompt 4 Test Accuracy: 1.0
Prompt 5 Train Accuracy: 1.0
Prompt 5 Test Accuracy: 1.0
Prompt 6 Train Accuracy: 1.0
Prompt 6 Test Accuracy: 1.0
Prompt 7 Train Accuracy: 1.0
Prompt 7 Test Accuracy: 1.0
Prompt 8 Train Accuracy: 1.0
Prompt 8 Test Accuracy: 1.0
Prompt 9 Train Accuracy: 1.0
Prompt 9 Test Accuracy: 1.0
Prompt 10 Train Accuracy: 1.0
Prompt 10 Test Accuracy: 1.0


217

In [10]:
# print(type(preds))

In [11]:
# print(type(y_test))